In [5]:
import os
import pathlib

from openai import OpenAI

with open(".env", "r") as f:
    os.environ.update(
        dict(line.strip().split("=") for line in f if not line.startswith("#"))
    )

client = OpenAI()


message = "Hello, how are you today?"
message = "".join("\u00a0" if i % 2 else c for i, c in enumerate(message))


completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": "The following is a conversation with an AI assistant. The assistant is helpful, creative, clever, and very friendly.",
        },
        {"role": "user", "content": message},
    ],
)
print(completion.choices[0].message.content)
if "\u00a0" in completion.choices[0].message.content:
    print("Success")
else:
    print("Failure")


Hello! It looks like you had a bit of fun with your keyboard. How can I assist you today?
Failure


In [11]:
from datasets import load_dataset

corpus = load_dataset("MarkrAI/AutoRAG-evaluation-2024-LLM-paper-v1", "corpus")
qa = load_dataset("MarkrAI/AutoRAG-evaluation-2024-LLM-paper-v1", "qa")

print(len(corpus["train"]), len(qa["train"]))
print(corpus["train"][0])
print(corpus["train"][0]["contents"])

8576 520
{'doc_id': '6f86094c-47fe-43de-a77a-e8c34c69c997', 'contents': "# Rag-Driver: Generalisable Driving Explanations With Retrieval-Augmented In-Context Learning In Multi-Modal Large Language Model\n\nJianhao Yuan1, Shuyang Sun1, Daniel Omeiza1, Bo Zhao2, Paul Newman1, Lars Kunze1, Matthew Gadd1\n1 University of Oxford 2 Beijing Academy of Artificial Intelligence\n{jianhaoyuan,kevinsun,daniel,pnewman,lars,mattgadd}@robots.ox.ac.uk  \nAbstract—Robots powered by 'blackbox' models need to provide\nhuman-understandable explanations which we can trust. Hence,\nexplainability plays a critical role in trustworthy autonomous\ndecision-making to foster transparency and acceptance among\nend users, especially in complex autonomous driving. Recent\nadvancements in Multi-Modal Large Language models (MLLMs)\nhave shown promising potential in enhancing the explainability\nas a driving agent by producing control predictions along with\nnatural language explanations. However, severe data scarcity

In [7]:
res = client.embeddings.create(
    input=corpus["train"][0]["contents"],
    model="text-embedding-3-small",
)
print(res.data[0].embedding)

[0.021136324852705002, 0.013456235639750957, 0.0002930687041953206, -0.018885042518377304, 0.007795868907123804, -0.03542875126004219, -0.004862769972532988, 0.041886717081069946, -0.024120882153511047, 0.03411657363176346, 0.024416765198111534, -0.06879919022321701, -0.023953644558787346, 0.015077158808708191, -0.0007935769972391427, -0.03756425157189369, 0.006461180280894041, -2.7940021027461626e-05, -0.00343481358140707, 0.019296705722808838, 0.044588252902030945, -0.016260690987110138, 0.018782125785946846, 0.004566886927932501, -0.0067795757204294205, -0.04667229950428009, 0.04528293386101723, 0.02498280070722103, 0.008863620460033417, -0.031132018193602562, 0.03375637158751488, -0.033421892672777176, -0.037255506962537766, -0.012954521924257278, -0.028867870569229126, 0.020107166841626167, 0.017161203548312187, 0.009056586772203445, -0.020017115399241447, -0.0013161961687728763, 0.01413805317133665, -0.036457907408475876, -0.00044945243280380964, 0.027684340253472328, 0.019631180

In [36]:
from typing import Iterable

from chromadb import EphemeralClient
from chromadb.config import Settings
from chromadb.types import Collection
from chromadb.utils.embedding_functions.openai_embedding_function import (
    OpenAIEmbeddingFunction,
)

try:
    api_key = os.environ["OPENAI_API_KEY"]
except KeyError:
    raise ValueError("No API key.")

chroma_client = EphemeralClient(Settings(anonymized_telemetry=False))
embedding_fn = OpenAIEmbeddingFunction(api_key, model_name="text-embedding-3-small")

papers_collection = chroma_client.create_collection(
    name="papers", embedding_function=embedding_fn, get_or_create=True
)


def embed(
    coll: Collection, docs: Iterable[str], ids: Iterable[str], batch_size: int = 2048
) -> None:
    """Embeds a list of documents and update or insert them into a collection."""
    for i in range(0, len(docs), batch_size):
        coll.upsert(documents=docs[i : i + batch_size], ids=ids[i : i + batch_size])

In [37]:
embed(papers_collection, corpus["train"]["contents"], corpus["train"]["doc_id"])

In [38]:
papers_collection.query(query_texts=["Hello, how are you today?"])

{'ids': [['a75255b0-9a26-46e9-bcd2-5347ccdb8faf',
   '5d408ddd-eebc-4c7b-a8ac-0456e72ae844',
   '4ebc3bca-c367-4a3d-8a93-f09b808cc1c7',
   'fb0e3fa8-5f44-4cfe-a1d2-586396993062',
   '61e68a17-14d1-4c10-b8e1-a2be36667a48',
   '1c422184-0d91-43d6-8d3e-1f2dedd7ed4c',
   'a93a3b06-fd76-45e7-99ac-9a8b6dac4055',
   '1428e922-bc06-4c79-940f-e243e6881e5d',
   '33e14682-afef-4259-af33-296e08039186',
   'b821b032-2c66-43ae-bc99-5a7e56454549']],
 'embeddings': None,
 'documents': [["## Gpt-3.5-Turbo'S And Openchat-3.5'S System Message\n\nYou are a helpful assistant.",
   '## B Crafted Examples\n\n                   | 0701                    |           47 | .                |\n| OpenChat 3.5                                                                |                               |                         |              |                  |\n| 10                                                                          |                               |                         |              |

In [58]:
for i, q in enumerate(qa["train"][:100]["query"]):
    print(i, q)

0 Does the Turing Test assess a machine's ability to exhibit intelligent behavior equivalent to that of a human?
1 Is the oa_temp or the zone_occ the most impactful feature according to the Shapley values?
2 What is the performance percentage increase observed in the navigational prompt suffix attack scenario when using VELMA-FT?
3 What are essential components of evaluating large language models (LLMs)?
4 How many errors were there in the Inference phase as per the document?
5 What is the meaning of PLP-former in the context of this text?
6 How does the Qmsum value vary between the first and the third entries?
7 What is the purpose of using the torch.einsum function in the provided text?
8 What is the result of applying the PAL attack against GPT-3.5-Turbo?
9 What is the process to deceive the test function as described?
10 What leads to a greater decrease in performance in deploying LLMs/VLMs in robotics according to the experiment?
11 Why has the accuracy of the models not reached 1

In [ ]:
model = "gpt-3.5-turbo"

question = qa["train"][88]["query"]
answer = qa["train"][88]["generation_gt"][0]

context = "\n\n".join(
    "\n".join(doc)
    for doc in papers_collection.query(query_texts=[question], n_results=1)["documents"]
)
message = question


# message = f"""Question:
# {question}

# Context:
# {context}"""
print(f"Question: {question}")
print(f"Dateset answer: {answer}")

completion_rag = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant, answer in one sentence.",
        },
        {"role": "assistant", "content": context},
        {"role": "user", "content": message},
    ],
)

print(f"Model answer with context: {completion_rag.choices[0].message.content}")

completion_norag = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant, answer in one sentence.",
        },
        {"role": "user", "content": message},
    ],
)

print(f"Model answer without context: {completion_norag.choices[0].message.content}")


Question: What makes the GRACE framework able to perform generative cross-modal retrieval?
Dateset answer: GRACE assigns unique identifiers to images and comprises two training steps: learning to memorize the associations between visual content and their identifiers, and learning to retrieve the identifier of a relevant image given a textual query.
Model answer with context: GRACE assigns unique identifiers to images, facilitating visual memory and enabling the model to generate identifiers as retrieval results, thus allowing it to effectively memorize and recall images in response to textual queries.
Model answer without context: The GRACE framework uses graph-based reasoning and contrastive learning to effectively generate and retrieve cross-modal data.


In [66]:
print(context)

ids

embeddings

documents

uris

data

metadatas

distances

included


In [ ]:
from ragas.metrics import BleuScore, RougeScore
from ragas import SingleTurnSample

bleu = BleuScore()
rouge = RougeScore()
test_data = SingleTurnSample(
    user_input=
)


